In [29]:
import json
import importlib
from nba_api.stats import endpoints


# Sample parameters for testing
SAMPLE_PARAMS = {
    'game_id': '0022300001',
    'player_id': '2544',  # LeBron James
    'team_id': '1610612747',  # Lakers
    'season': '2023-24',
    'season_type': 'Regular Season'
}


In [88]:
# Analysis: Find all endpoints whose priority is not 'None' and group by required parameters
import json
from collections import defaultdict

# Load the current endpoint configuration
with open('endpoint_priority_review.json', 'r') as f:
    config = json.load(f)

# Count endpoints by different criteria
total_endpoints = len(config['endpoints'])
priority_none = 0
priority_not_none = 0
latest_version_true = 0
latest_version_false = 0

# Analyze each endpoint and group by required params (only non-None priority AND no policy)
active_endpoints = []
active_endpoints_by_params = defaultdict(list)
no_policy_count = 0

for endpoint_name, endpoint_info in config['endpoints'].items():
    # Count priority status
    if endpoint_info.get('priority') == 'None':
        priority_none += 1
    else:
        priority_not_none += 1
        
        # Check if policy is not set (None, empty string, or missing)
        policy = endpoint_info.get('policy')
        has_no_policy = not policy or policy == 'None' or policy == ''
        
        if has_no_policy:
            no_policy_count += 1
            active_endpoints.append(endpoint_name)
            
            # Group by required parameters (only for non-None priority AND no policy endpoints)
            required_params = endpoint_info.get('required_params', [])
            param_key = tuple(sorted(required_params)) if required_params else ()
            active_endpoints_by_params[param_key].append(endpoint_name)
    
    # Count latest_version status
    if endpoint_info.get('latest_version') is True:
        latest_version_true += 1
    else:
        latest_version_false += 1

print("=== ACTIVE ENDPOINTS ANALYSIS ===")
print(f"Total endpoints: {total_endpoints}")
print()
print("PRIORITY STATUS:")
print(f"  priority = 'None':     {priority_none}")
print(f"  priority ≠ 'None':     {priority_not_none}")
print()
print("LATEST VERSION STATUS:")
print(f"  latest_version = True:  {latest_version_true}")
print(f"  latest_version = False: {latest_version_false}")
print()
print("🎯 ACTIVE ENDPOINTS (priority ≠ 'None' AND no policy set):")
print(f"  Count: {no_policy_count}")
print(f"  Percentage: {no_policy_count/total_endpoints*100:.1f}%")

print(f"\n📊 ACTIVE ENDPOINTS (NO POLICY) GROUPED BY REQUIRED PARAMETERS:")
print(f"Found {len(active_endpoints_by_params)} different parameter combinations")

# Sort groups by number of endpoints (most common first)
sorted_param_groups = sorted(active_endpoints_by_params.items(), key=lambda x: len(x[1]), reverse=True)

for param_tuple, endpoints in sorted_param_groups:
    if not param_tuple:
        param_str = "NO PARAMETERS"
    else:
        param_str = ", ".join(param_tuple)
    
    print(f"\n📋 REQUIRES: [{param_str}]")
    print(f"   Count: {len(endpoints)} endpoints")
    
    # Show all endpoints in this group
    for i, endpoint in enumerate(sorted(endpoints)):
        # Get priority and latest_version info
        endpoint_info = config['endpoints'][endpoint]
        priority = endpoint_info.get('priority', 'unknown')
        is_latest = endpoint_info.get('latest_version', False)
        latest_icon = "✅" if is_latest else "❌"
        policy = endpoint_info.get('policy', 'None')
        
        print(f"   {i+1:2d}. {endpoint} (priority: {priority}, policy: {policy}) {latest_icon}")

print(f"\n🎯 SUMMARY: {no_policy_count} active endpoints with no policy set - ready for policy assignment!")

=== ACTIVE ENDPOINTS ANALYSIS ===
Total endpoints: 136

PRIORITY STATUS:
  priority = 'None':     72
  priority ≠ 'None':     64

LATEST VERSION STATUS:
  latest_version = True:  125
  latest_version = False: 11

🎯 ACTIVE ENDPOINTS (priority ≠ 'None' AND no policy set):
  Count: 1
  Percentage: 0.7%

📊 ACTIVE ENDPOINTS (NO POLICY) GROUPED BY REQUIRED PARAMETERS:
Found 1 different parameter combinations

📋 REQUIRES: [player_id]
   Count: 1 endpoints
    1. PlayerNextNGames (priority: unknown, policy: ) ✅

🎯 SUMMARY: 1 active endpoints with no policy set - ready for policy assignment!


In [ ]:
from nba_api.stats.endpoints import PlayerNextNGames      
PlayerNextNGames(player_id=SAMPLE_PARAMS['player_id']).get_data_frames()[0]

,GROUP_SET,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,VS_PLAYER_ID,VS_PLAYER_NAME,COURT_STATUS,GP,MIN,PLUS_MINUS,OFF_RATING,DEF_RATING,NET_RATING
0,On/Off Court,1610612747,LAL,Los Angeles Lakers,1631108,"Christie, Max",On,46,1154.0,-0.7,111.9,112.7,-0.9
1,On/Off Court,1610612747,LAL,Los Angeles Lakers,203076,"Davis, Anthony",On,42,1439.0,0.7,113.8,113.2,0.5
2,On/Off Court,1610612747,LAL,Los Angeles Lakers,1629029,"Dončić, Luka",On,28,984.0,8.6,120.0,111.5,8.4
3,On/Off Court,1610612747,LAL,Los Angeles Lakers,1627827,"Finney-Smith, Dorian",On,43,1239.0,10.4,120.2,109.1,11.1
4,On/Off Court,1610612747,LAL,Los Angeles Lakers,1630692,"Goodwin, Jordan",On,29,543.0,-1.0,111.2,112.4,-1.2
5,On/Off Court,1610612747,LAL,Los Angeles Lakers,1629060,"Hachimura, Rui",On,59,1869.0,4.1,114.8,110.9,3.9
6,On/Off Court,1610612747,LAL,Los Angeles Lakers,1629637,"Hayes, Jaxson",On,56,1093.0,3.8,112.3,109.8,2.5
7,On/Off Court,1610612747,LAL,Los Angeles Lakers,1641720,"Hood-Schifino, Jalen",On,2,14.0,27.2,115.2,100.0,15.2
8,On/Off Court,1610612747,LAL,Los Angeles Lakers,1642355,"James, Bronny",On,27,181.0,-18.8,95.5,114.1,-18.6
9,On/Off Court,1610612747,LAL,Los Angeles Lakers,2544,"James, LeBron",On,70,2444.0,-1.1,112.7,114.0,-1.3
